## Pandas Profiling: USA Air Pollution Data
Source of data: https://data.world/data-society/us-air-pollution-data

The autoreload instruction reloads modules automatically before code execution, which is helpful for the update below.

In [2]:
%load_ext autoreload
%autoreload 2

Make sure that we have the latest version of pandas-profiling.

In [3]:
%%capture
import sys

%pip install -U ydata_profiling pandas jupyter ipywidgets json numpy pathlib

# Skip this for now, we'll create HTML reports instead
# !jupyter nbextension enable --py widgetsnbextension

# You might want to restart the kernel now.

### Import libraries

In [4]:
import pandas as pd

from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file
from ydata_profiling.config import Dataset
import json
from pathlib import Path

import numpy as np
import pandas as pd


/tmp/ipykernel_10288/1250497162.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:


file_name = cache_file(
    "census_train.csv",
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
)

# Names based on https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
df = pd.read_csv(
    file_name,
    header=None,
    index_col=False,
    names=[
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
    ],
)

# Prepare missing values
df = df.replace("\\?", np.nan, regex=True)

# Initialize the report
profile = ProfileReport(df, title="Census Dataset", explorative=True)

# show column definition
with open("census_column_definition.json") as f:
    definitions = json.load(f)

profile.config.dataset = Dataset(
    description='Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0)). Prediction task is to determine whether a person makes over 50K a year.',
    copyright_year="1996",
    author="Ronny Kohavi and Barry Becker",
    creator="Barry Becker",
    url="https://archive.ics.uci.edu/ml/datasets/adult",
)
profile.config.variables.descriptions = definitions

# Only show the descriptions in the overview
profile.config.show_variable_description = False

profile.to_file(Path("./census_report.html"))

/tmp/ipykernel_10288/770057977.py:7: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/codespace/.python/current/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: ' Bachelors'')
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/home/codespace/.python/current/lib/python3.10/site-packages/ydata_profiling/model/missing.py:78: UserWarning: There was an attempt to generate the Heatmap missing values diagrams, but this failed.
To hide this warni

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Load and prepare the dataset

In [ ]:
file_name = cache_file(
    "pollution_us_2000_2016.csv",
    "https://query.data.world/s/mz5ot3l4zrgvldncfgxu34nda45kvb",
)

df = pd.read_csv(file_name, index_col=[0])

# We will only consider the data from Arizone state for this example
df = df[df["State"] == "Arizona"]
df["Date Local"] = pd.to_datetime(df["Date Local"])

### Multi-entity time-series

The support to time series can be enabled by passing the parameter tsmode=True to the ProfileReport when its enabled, pandas profiling will try to identify time-dependent features using the feature's autocorrelation, which requires a sorted DataFrame or the definition of the `sortby` parameter.

When a feature is identified as time series will trigger the following changes:
   - the histogram will be replaced by a line plot
   - the feature details will have a new tab with autocorrelation and partial autocorrelation plots
   - two new warnings: `NON STATIONARY` and `SEASONAL` (which indicates that the series may have seasonality)

In cases where the data has multiple entities,  as in this example, where we have different meteorological stations, each station can be interpreted as a time series, its necessary to filter the entities and profile each station separately.

The following plot showcases the amount of data for each entity over time. In this case the data from the stations started being collected at the same period, and the data is collected hourly so they have the same amount of data per period.

# VS Code may require `Jupyter Notebook Renderers` extension

In [ ]:
from ydata_profiling.visualisation.plot import timeseries_heatmap

timeseries_heatmap(dataframe=df, entity_column="Site Num", sortby="Date Local")

In [ ]:
# # Return the profile per station
# for group in df.groupby("Site Num"):
#     # Running 1 profile per station
#     profile = ProfileReport(
#         group[1],
#         tsmode=True,
#         sortby="Date Local",
#         # title=f"Air Quality profiling - Site Num: {group[0]}"
#     )

#     profile.to_file(f"Ts_Profile_{group[0]}.html")

In [5]:
file_name = cache_file(
    "census_train.csv",
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
)

# Names based on https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
df = pd.read_csv(
    file_name,
    header=None,
    index_col=False,
    names=[
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
    ],
)

# Prepare missing values
df = df.replace("\\?", np.nan, regex=True)

# Initialize the report
profile = ProfileReport(df, title="Census Dataset", explorative=True)

# show column definition
with open("census_column_definition.json") as f:
    definitions = json.load(f)

profile.config.dataset = Dataset(
    description='Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0)). Prediction task is to determine whether a person makes over 50K a year.',
    copyright_year="1996",
    author="Ronny Kohavi and Barry Becker",
    creator="Barry Becker",
    url="https://archive.ics.uci.edu/ml/datasets/adult",
)
profile.config.variables.descriptions = definitions

# Only show the descriptions in the overview
profile.config.show_variable_description = False

profile.to_file(Path("./census_report.html"))

/tmp/ipykernel_8769/3031617755.py:7: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(


FileNotFoundError: [Errno 2] No such file or directory: 'census_column_definition.json'